In [ ]:
import numpy as np

from pydrake.all import (AddMultibodyPlantSceneGraph,
                         DiagramBuilder,
                         RigidTransform,
                         BsplineTrajectory,
                         KinematicTrajectoryOptimization,
                         SnoptSolver)

from manipulation.scenarios import (AddIiwa, AddWsg)

In [ ]:
# Joint position and velocity constraints
def make_kin_traj_opt_trajectory(q_Throw, q_Catch, v_Throw, v_Catch, t_duration):
    # Set up plant
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    gripper_frame = plant.GetFrameByName("body", wsg)
    plant.Finalize()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # Initialize KinTrajOpt
    num_positions = 7
    num_control_points = 21
    trajopt = KinematicTrajectoryOptimization(num_positions, num_control_points)
    prog = trajopt.get_mutable_prog()

    # Provide Initial Guess
    q_guess = np.tile(q_Catch.reshape((num_positions,1)), (1, trajopt.num_control_points()))
    q_guess[0,:] = np.linspace(q_Catch[0], q_Throw[0], trajopt.num_control_points())
    path_guess = BsplineTrajectory(trajopt.basis(), q_guess)
    trajopt.SetInitialGuess(path_guess)
    
    # Add Position and Velocity Bounds based on iiwa
    trajopt.AddPositionBounds(plant.GetPositionLowerLimits()[:num_positions],
                              plant.GetPositionUpperLimits()[:num_positions])
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits()[:num_positions],
                              plant.GetVelocityUpperLimits()[:num_positions])
    
    # Constrain duration to be exact duration
    trajopt.AddDurationConstraint(2 * t_duration, 2 * t_duration)

    # Constrain positions in joint angles
    trajopt.AddPathPositionConstraint(q_Catch, q_Catch, 0)
    trajopt.AddPathPositionConstraint(q_Throw, q_Throw, 0.5)
    trajopt.AddPathPositionConstraint(q_Catch, q_Catch, 1)

    # Add Costs
    trajopt.AddPathLengthCost(1)

    prog.AddQuadraticErrorCost(np.eye(7), q_Catch,
                               trajopt.control_points()[:, 0])
    
    prog.AddQuadraticErrorCost(np.eye(7), q_Throw,
                               trajopt.control_points()[:, np.round(num_control_points/2).astype(int)])

    prog.AddQuadraticErrorCost(np.eye(7), q_Catch,
                               trajopt.control_points()[:, -1])

    trajopt.AddPathVelocityConstraint(v_Catch,v_Catch, 0)
    trajopt.AddPathVelocityConstraint(v_Throw,v_Throw, 0.5)
    trajopt.AddPathVelocityConstraint(v_Catch,v_Catch, 1)


    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not result.is_success():
        print("Trajectory optimization failed!")
        print(result.get_solver_id().name())
    else:
        print("Trajectory optimization succeeded!")
    
    pos_traj = trajopt.ReconstructTrajectory(result)
    vel_traj = pos_traj.MakeDerivative()
    acc_traj = vel_traj.MakeDerivative()
    
    return pos_traj, vel_traj, acc_traj

In [ ]:
t_duration = 0.782
q_Catch = np.array([-0.74, 0.58, 0, -1.79, 0, -0.79, 0])
q_Throw = np.array([-0.54, 0.58, 0, -1.79, 0, -0.79, 0])
v_Catch = np.array([[-0.151], [3.974], [-0.121], [-0.673],[0.039], [1.305], [-0.000]])
v_Throw = np.array([[-0.151], [-4.050], [-0.121], [0.559], [0.039], [-1.322], [0.000]])

pos_traj, vel_traj, acc_traj = make_kin_traj_opt_trajectory(q_Throw, q_Catch, v_Throw, v_Catch, t_duration)

print(f"traj vel at time 0:\n{vel_traj.value(0).flatten()}")
print(f"joint velocities set as constraint at time 0:\n{v_Catch.flatten()}")
print()
print(f"traj vel at time {t_duration}:\n{vel_traj.value(t_duration).flatten()}")
print(f"joint velocities set as constraint at time {t_duration}:\n{v_Throw.flatten()}")
print()
print(f"traj vel at time {2*t_duration}:\n{vel_traj.value(2*t_duration).flatten()}")
print(f"joint velocities set as constraint at time {2*t_duration}:\n{v_Catch.flatten()}")

Trajectory optimization succeeded!
traj vel at time 0:
[-0.09654731  2.54092072 -0.07736573 -0.43030691  0.02493606  0.83439898
  0.        ]
joint velocities set as constraint at time 0:
[-0.151  3.974 -0.121 -0.673  0.039  1.305 -0.   ]

traj vel at time 0.782:
[-0.09654731 -2.58951407 -0.07736573  0.35741688  0.02493606 -0.84526854
  0.        ]
joint velocities set as constraint at time 0.782:
[-0.151 -4.05  -0.121  0.559  0.039 -1.322  0.   ]

traj vel at time 1.564:
[-0.09654731  2.54092072 -0.07736573 -0.43030691  0.02493606  0.83439898
  0.        ]
joint velocities set as constraint at time 1.564:
[-0.151  3.974 -0.121 -0.673  0.039  1.305 -0.   ]


In [ ]:
t_duration = 0.452
q_Catch = np.array([-0.74, 0.58, 0, -1.79, 0, -0.79, 0])
q_Throw = np.array([-0.54, 0.58, 0, -1.79, 0, -0.79, 0])
v_Catch = np.array([[-0.261], [2.250], [-0.210], [-0.455], [0.067], [0.743], [-0.000]])
v_Throw = np.array([[-0.261], [-2.382], [-0.210], [0.257], [0.067], [-0.773], [0.000]])

pos_traj, vel_traj, acc_traj = make_kin_traj_opt_trajectory(q_Throw, q_Catch, v_Throw, v_Catch, t_duration)

print(f"traj vel at time 0:\n{vel_traj.value(0).flatten()}")
print(f"joint velocities set as constraint at time 0:\n{v_Catch.flatten()}")
print()
print(f"traj vel at time {t_duration}:\n{vel_traj.value(t_duration).flatten()}")
print(f"joint velocities set as constraint at time {t_duration}:\n{v_Throw.flatten()}")
print()
print(f"traj vel at time {2*t_duration}:\n{vel_traj.value(2*t_duration).flatten()}")
print(f"joint velocities set as constraint at time {2*t_duration}:\n{v_Catch.flatten()}")

Trajectory optimization succeeded!
traj vel at time 0:
[-0.28871681  2.48893805 -0.23230088 -0.50331858  0.07411504  0.82190265
  0.        ]
joint velocities set as constraint at time 0:
[-0.261  2.25  -0.21  -0.455  0.067  0.743 -0.   ]

traj vel at time 0.452:
[-0.28871681 -2.63495575 -0.23230088  0.28429204  0.07411504 -0.8550885
  0.        ]
joint velocities set as constraint at time 0.452:
[-0.261 -2.382 -0.21   0.257  0.067 -0.773  0.   ]

traj vel at time 0.904:
[-0.28871681  2.48893805 -0.23230088 -0.50331858  0.07411504  0.82190265
  0.        ]
joint velocities set as constraint at time 0.904:
[-0.261  2.25  -0.21  -0.455  0.067  0.743 -0.   ]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68a898be-bd4e-4046-916a-6293aadf6c5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>